## Извлечение ключевых слов

Или keyword extraction - это упрощенный вариант задачи саммаризации (иногда это называется реферирование, но мне не очень нравится), т.е. извлечения главной информации из текста. Настоящая саммаризация подразумевает, что главная информация описывается нормальными полными предложениями, но сделать это очень сложно. Извлечь только основные слова проще и задачу решает тоже достаточно хорошо (по нескольким слова сразу приблизительно понятно о чем текст и читать ключевые слова сильно быстрее, чем даже самое хороше саммари). Еще одно преимущество ключевых слов - это то, что их удобно использовать в стандартном поиске (который работает только со словами и не анализирует последовательности). 

In [2]:
from platform import python_version

print(python_version())

3.6.5


Одному тексту разные люди могут приписать разные ключевые слова, поэтому оценивать извлечение ключевых слов сложнее, чем обычную классификации. Нужно либо для каждого текста иметь несколько набор ключевых слов для каждого текста, либо брать тексты с ключевыми словами из разных источников. В любом случае с метрикам нужно быть аккуратными и смотреть больше на соотношение, чем на абсолютные значения. Ну и проверять полученные алгоритмы в реальной задаче или хотя бы на глаз. 

In [3]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [4]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [5]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = 'data'

In [16]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [21]:
for index,file in enumerate(files):
    if 'DS_Store' in file:
        files.pop(index)

In [22]:
files

['data/ng_1.jsonlines', 'data/ng_0.jsonlines']

Объединим файлы в один датасет.

In [24]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [27]:
data.shape

(1987, 5)

In [30]:
type(data)

pandas.core.frame.DataFrame

In [31]:
data.head(10)

,content,keywords,summary,title,url
0,"В среду состоялось отложенное заседание Совета по федеральным государственным образовательным стандартам (ФГОС) при Министерстве образования и науки РФ. Собрание должно было состояться еще в понедельник, но было перенесено по просьбе членов совета. И вот пришло сообщение, что общественники выразили согласие с позицией министерства. Новые ФГОСы приняты.\nНа вчерашнем заседании был принят ФГОС по начальной общеобразовательной школе. До 28 марта продлятся косультации по ФГОСам для средней школы.\nНапомним, что накануне Гильдия словесников разместила открытое письмо на имя министра образования и науки РФ Ольги Васильевой. По мнению авторов письма, новые ФГОСы грубо нарушают права детей, уже проучившихся по существующему стандарту до 6-го класса. Приняв новый стандарт, Министерство образования дает право контролирующим органам ловить детей на незнании большого списка произведений (235 за пять лет обучения). «Это исключает возможность полноценного их освоения, создает риск формального, п...","[школа, образовательные стандарты, литература, история, фгос]","Глава Минобрнауки считает, что в нездоровом ажиотаже вокруг новых образовательных стандартов виноваты издательства учебной литературы","Ольга Васильева обещала ""НГ"" не перегружать школьников",https://amp.ng.ru/?p=http://www.ng.ru/education/2018-03-22/8_7195_school.html
1,"Хорошо, когда красота в глазах смотрящего живет свободно или хотя бы занимает широкий угол зрения. Плохо было б, если б она вовсе не озаряла своим светом космическую темень пустоты зрачка. Слава богу, такое вряд ли возможно. \nА случается, что красота уходит. Почему вдруг? И куда она девается, когда в один из философских обходов своего организма вы, еще недавно гордый ее обладатель, обескураженно ее недосчитываетесь? \nВообразите: прелестнейшее из созданий – ваша кошка пластичнейшими движениями рвет банкноту за банкнотой, забирается на карниз по шелковой занавеске или отгрызает полпаспорта. Где, скажите, теперь красота этой кошки? Или другой пример – с зазнобой сердца. Предмет романтичнейших грез наконец-то садится с вами на заветную скамейку в парке – закат, пение птах… И тут он силой своего обаяния с оглушительным плюхом обрушивает вокруг вас красоту и гармонию столетних дубов, тополей и прочего. Где, спрашивается, красота момента? \nЕсли от сказки после того, как ее рассказали,...","[красота, законы]",О живительной пользе укорота при выборе между плохим и хорошим,У красоты собственные закон и воля,https://amp.ng.ru/?p=http://www.ng.ru/style/2018-03-19/8_7192_beauty.html
2,"Когда-то Леонид Юзефович написал книгу о монгольской эпопее барона Унгерна «Самодержец пустыни» – она стала интеллектуальным бестселлером и классикой жанра – документальный роман. В то время автор попутно изучал и историю вооруженного восстания в Якутии в 1922–1923 годах под руководством Анатолия Пепеляева. И вот теперь из «якутского» материала сложилась отдельная книга. Тема ее для нынешнего читателя поистине раритетна. Ведь воевавший где-то на самом краю страны Пепеляев практически забыт, притом что о борьбе с ним когда-то в СССР выходили статьи и книги. В памяти потомков, образно говоря, от Пепеляева остался только пепел.\nЮзефович воскрешает в памяти не только его военные дела, но и человеческие черты. Этот провинциальный интеллигент, неврастеник и фаталист, начал восстание, практически не имея шансов на успех. Однако силою недюжинной харизмы Пепеляев сумел собрать вокруг себя многих боевых офицеров, таежных охотников и недовольных новыми порядками аборигенов. Для своих 32 лет ...","[юзефович, гражданская война, пепеляев, якутия]",Крепость из тел и призрак независимой Якутии,Апокалиптический бунт,https://amp.ng.ru/?p=http://www.ng.ru/zavisimaya/2017-12-19/15_7139_bunt.html
3,"Гран-При Испании открыло евротур «Формулы-1». Гонка на трассе близ Барселоны пока остается в календаре как минимум до 2019 года, чего нельзя сказать о других автодромах Старого Света. Тем не менее, в глаза резко бросались полу

In [51]:
data.iloc[[16,30],]

,content,keywords,summary,title,url
16,"По критериям, принятым в развитых странах, состояние водного хозяйства Российской Федерации следует характеризовать как кризисное. Это связано не с количеством вод, а с их качеством. Услугами централизованного водоснабжения пользуются 75% населения, но в малых городах, поселках городского типа, сельских населенных пунктах этот показатель и того меньше – 60%. В развитых странах эта цифра превышает 90–95%. Очистку в системах водоподготовки РФ проходит не более 59% сырой воды, в сельских пунктах – менее 20%. В результате каждый второй житель нашей страны использует для питья воду, не соответствующую установленным нормативам по ряду показателей. Это угрожает безопасности России.\nВ значительной степени такая ситуация обусловлена кризисами 1990-х и 2000-х годов. Инвестиции в водное хозяйство все эти годы были явно недостаточными. Текущие затраты на охрану и рациональное использование водных ресурсов в 2015 году по сравнению с 2005-м снизились на 15% в сопоставимых ценах.\nТак, в 2005–20...","[вода, экология, экономика, водное пространство, гост, качество]","Каждый второй россиянин пьет воду, не соответствующую ГОСТам качества",Девятый вал проблем водоочистки,http://www.ng.ru/ng_ekologiya/2017-10-11/11_7092_problem.html
30,"Прежде всего необходимо констатировать, что санкции постепенно вымывают из экономического сотрудничества России с Западом все новые и новые области деятельности. Российская промышленность ощущает острую нехватку ноу-хау и инвестиций, поскольку под запреты подпадают уже практически все наукоемкие производства.\nДавайте вспомним, что в конце января Минфин США представил «кремлевский доклад» – список официальных лиц и бизнесменов, которых администрация в Вашингтоне считает приближенными к президенту России Владимиру Путину и причастными к коррупции. В документе перечислено в общей сложности 210 имен, среди которых 114 – имена членов высшего политического руководства РФ и 96 олигархов. В список среди прочих попали премьер-министр Дмитрий Медведев и все члены кабинета министров, а также 43 сотрудника администрации президента РФ, включая пресс-секретаря Дмитрия Пескова, главу администрации Антона Вайно, советников и помощников главы государства. Кроме того, после публикации первой части...","[экология, зеленые технологии, россия, германия]",Экономический ущерб от нерешенных экологических проблем в РФ достигает 6% ВВП,Зеленые технологии в рамках санкционной борьбы,http://www.ng.ru/ng_energiya/2018-03-20/9_7193_sanctions.html


## Каждой статье приписано какое-то количество ключевых слов. **Наша задача - придумать как извлекать точно такой же список автоматически.**
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [57]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [10]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем долго думать и тестрировать первое, что приходит в голову.

Возьмем первые 5 слов из заголовка.

In [11]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [12]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [13]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [14]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [15]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [16]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                      [в, и, на, не, что, –, его, «молодежное, с, это]
1                                                            [в, и, на, –, млрд., куб., по, к, газа, м]
2                                                                 [в, –, и, не, я, но, что, это, на, с]
3                                                       [в, на, и, ким, по, –, что, видео, он, зинаиды]
4                                              [в, и, на, новой, площадью, москвы, –, развития, с, для]
5                                                             [в, на, и, не, с, но, уже, что, у, гонки]
6                                                  [на, в, (с, место)., и, рф, позиции, влияние, по, с]
7                                                        [в, и, –, по, с, культуре, не, из, будет, как]
8                                                              [в, и, на, с, что, для, по, –, не, газа]
9    [в, рф, террористической, организации, задержаны, –, четвер

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [17]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return words

In [18]:
data['content_norm'] = data['content'].apply(normalize)

In [19]:
data['title_norm'] = data['title'].apply(normalize)

In [20]:
data['title_norm'].head(10)

0            [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                 [газпром, хватить]
2                                      [бесконечный, партия, четырехмерный, шахматы]
3    [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                 [новый, москва, остаться, территория, экологический, безопасность]
5                         [f1, гран-при, сша, пройти, четыре, машина, стопка, штраф]
6                                [100, ведущий, политик, россия, февраль, 2018, год]
7                                           [закон, культура, принимать, фон, арест]
8                     [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                       [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [21]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [22]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


In [38]:
data['title_norm'].apply(lambda x: x[:10])

0                               [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                                    [газпром, хватить]
2                                                         [бесконечный, партия, четырехмерный, шахматы]
3                       [экс-депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                                    [новый, москва, остаться, территория, экологический, безопасность]
5                                            [f1, гран-при, сша, пройти, четыре, машина, стопка, штраф]
6                                                   [100, ведущий, политик, россия, февраль, 2018, год]
7                                                              [закон, культура, принимать, фон, арест]
8                                        [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                                          [фсб, калужский, обла

Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [23]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0                         [яблоко, молодёжный, который, акция, год, активист, это, деятельность, политика, наш]
1                               [миллиард, газа, год, куб, метр, газпром, добыча, 2020, должный, производитель]
2                                              [год, это, книга, роман, тот, писать, выйти, один, мир, перевод]
3                                    [ким, зинаида, видео, год, журналист, суд, дело, бывший, футиный, который]
4                         [площадь, территория, новый, москва, га, который, столица, тинао, парковый, развитие]
5                                  [гонка, который, команда, место, позиция, два, один, из-за, круг, чемпионат]
6                        [место, влияние, рф, позиция, глава, россия, президент, сергей, политический, рейтинг]
7                 [культура, закон, который, сфера, стд, разработать, концепция, проект, изменение, сообщество]
8                              [газопровод, сирия, год, турция, газа, россия, европа, катар, который, ту

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [24]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [25]:
data['content_norm'] = data['content'].apply(normalize)

In [45]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.1


In [37]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)])

0                     [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1                            [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                             [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                                  [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                                [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                                 [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6                            [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7                 [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                                 [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9

Ещу улучшения!

In [27]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0        [яблоко, акция, год, активист, деятельность, политика, власть, задача, молодая, человек]
1               [миллиард, газа, год, куб, метр, газпром, добыча, производитель, страна, прогноз]
2                [год, книга, роман, мир, перевод, стихотворение, читатель, жанр, поэзия, работа]
3                     [ким, зинаида, видео, год, журналист, суд, дело, рубль, процесс, заседание]
4                   [площадь, территория, москва, га, столица, тинао, развитие, парка, парк, год]
5                    [гонка, команда, место, позиция, круг, чемпионат, пилот, бокс, заезд, льюис]
6               [место, влияние, рф, позиция, глава, россия, президент, сергей, рейтинг, участие]
7    [культура, закон, сфера, концепция, проект, изменение, сообщество, услуга, учреждение, дело]
8                    [газопровод, сирия, год, турция, газа, россия, европа, катар, поток, проект]
9                 [участник, рф, организация, государство, область, центр, связь, фсб, март, год]
Name: content_norm, 

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [28]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [29]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [30]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [31]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [32]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [58]:
## так как матрица в tfidf в спарс формате,  ее нельзя просто так отсортировать
## перевести ее в обычный формат для всех данных тоже не получится - не хватит памяти
## поэтому пройдем по строчкам, переведем строчку в обычный array и отсортируем ее
keywords = []

for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])

In [59]:
keywords[:3]

[['яблоко',
  'активист',
  'акция',
  'дарья',
  'деятельность',
  'молодая человек',
  'политика',
  'виктор',
  'тимур',
  'репрессия'],
 ['миллиард куб',
  'куб метр',
  'куб',
  'газпром',
  'газа',
  'миллиард',
  'добыча',
  'добыча газа',
  'метр',
  'холдинг'],
 ['роман',
  'книга',
  'жанр',
  'стихотворение',
  'читатель',
  'перевод',
  'год',
  'поэзия',
  'произведение',
  'том']]

In [60]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за **baseline.**

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [46]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [89]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        
        if go_random:
            n = np.random.choice(len(vocab))
        
        
        ### 
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n
    


In [90]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

CPU times: user 1min 20s, sys: 274 ms, total: 1min 21s
Wall time: 1min 21s


In [91]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.21
F1 -  0.14
Jaccard -  0.08


In [92]:
keywords_rw.head(10)

0           [яблоко, год, жизнь, деятельность, человек, власть, мнение, акция, прошлое, дарья]
1            [миллиард, газпром, газа, куб, метр, год, производитель, добыча, прогноз, страна]
2             [год, герой, книга, роман, стих, том, стихотворение, поэзия, читатель, писатель]
3      [ким, видео, журналист, зинаида, свидетель, экспертиза, год, сентябрь, выборы, процесс]
4        [территория, площадь, развитие, москва, га, столица, парк, департамент, тинао, парка]
5                   [гонка, команда, позиция, место, бокс, фрэнк, победа, льюис, трасса, круг]
6    [место, влияние, рф, глава, президент, исследование, андрей, сергей, институт, политолог]
7     [культура, закон, дело, услуга, сфера, изменение, концепция, эрмитаж, срок, реставрация]
8          [газопровод, год, россия, газа, катар, поток, сирия, турция, статья, месторождение]
9               [участник, организация, рф, год, центр, март, связь, фсб, государство, житель]
Name: content_norm, dtype: object

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [93]:
import networkx as nx

In [94]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_array(matrix)
    # тут можно поставить любую метрику
    # менять тут 
    node2measure = dict(nx.degree_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [ ]:
RAKE 

Тут использован PageRank как метрика центральности. Про другие можно узнать вот тут - https://networkx.github.io/documentation/stable/reference/algorithms/centrality.html

Попробуйте разные метрики. Некоторые могут работать достаточно долго

In [95]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

CPU times: user 2min 8s, sys: 426 ms, total: 2min 9s
Wall time: 2min 9s


In [98]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09


Результаты не превосходят tfidf, но и не сильно уступают. Явно можно что-то доработать и превзойти baseline.

Готовое решение есть в gensim. Давайте попробуем его.

In [96]:
from gensim.summarization import keywords

In [97]:
gensim_kws = data['content_norm'].apply(lambda x: keywords(' '.join(x)).split('\n')[:10])

In [98]:
evaluate(data['keywords'], gensim_kws)

Precision -  0.07
Recall -  0.11
F1 -  0.08
Jaccard -  0.04


Наша имплементация отработала получше.

## Домашнее задание

В семинаре использовался только небольшой кусочек данных. На всех данных пересчитайте baseline (tfidf). 

**Ваша задача - предложить 3 способа побить бейзлайн на всех данных.**

Нет никаких ограничений кроме:

1) нельзя изменять метрику  
2) решение должно быть воспроизводимым  
3) способы дожны отличаться друг от друга не только гиперпараметрами (например, нельзя три раза поменять гиперпарамтры в TfidfVectorizer и сдать работу)  
4) изменение количества извлекаемых слов не является улучшением (выберите одно значение и используйте только его)  

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://forms.gle/GWzewBEpw8qnkv8t8

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другое.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 1 балл).

В поисках идей можно почитать обзоры по теме (посмотрите еще статьи, в которых цитируются эти обзоры): https://www.semanticscholar.org/search?year%5B0%5D=2012&year%5B1%5D=2020&publicationType%5B0%5D=Reviews&q=keyword%20extraction&sort=relevance

**Использовать доступные готовые решения тоже можно**. Так что погуглите перед тем, как приступать к заданию. 